***
## **<center>Predicting Book Award Candidates from Goodreads Data</center>**

### <center> **IST718 Final Project - Import and Merge Datasets</center>**

**<center>Jon Kaplan, Luke Miller, John Fields | June 13, 2020</center>**
***

<br>

The merging of the Books Fantasy-Paranormal, Reviews Fantasy-Paranormal and Awards would not run in Google Colab due to RAM limitations. This code was developed to run on local computers to generate a CSV file for import to IST718_Project_BookReviews notebook. Once this data is in Colab, the GPU's can be utilized for the BERT transformer processing.

Based on work by:
* University of California San Diego BookGraph - https://sites.google.com/eng.ucsd.edu/ucsdbookgraph


# 1. Data Import and Preprocessing

***

In [1]:
import numpy as np
import pandas as pd
import os
import json
import csv
from scipy.stats import uniform

# 1.1 Load Data from JSON files - BooksFP, ReviewsFP

#Source = University of California San Diego Book Graph

In [2]:
DIR = '/Users/johnfields/Library/Mobile Documents/com~apple~CloudDocs/Syracuse/IST718/Assignments/Project/UCSD Book Graph/'

In [3]:
#Load files function
#Limit to XXXX observations for testing
def load_data(file_name):
    count = 0
    data = []
    with open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)            
    return data
reviewsFP = load_data(os.path.join(DIR, 'goodreads_reviews_fantasy_paranormal.json'))   
booksFP = load_data(os.path.join(DIR, 'goodreads_books_fantasy_paranormal.json'))

## 1.1.0 Import and Load Award Data

In [0]:
## COMMENTED OUT SINCE THIS TAKES 25 MINUTES TO RUN
# #Import Award Data
# #04.25.20 Code From Luke Miller
# pd.options.display.max_columns = None
# logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

# def getAwardBooks(url, numPages, sleepTime):
#     logging.info('scraping {} pages from 1 to {}'.format(url,numPages))
#     all_awards = []
#     logging.info('loop through the other pages')
#     for i in range(1, numPages+1):
#         newPage = url+ '?page=' + str(i)
#         logging.info('logging page ' + str(i))
#         logging.info('url is {}'.format(newPage))
#         with urllib.request.urlopen(newPage) as response:
#             soup = BeautifulSoup(response, 'html.parser')
#             table = soup.find('table')

#         award_books = []
#         for i in table.findAll('tr',
#                                attrs={'itemtype': 'http://schema.org/Book'}):
#             rec = {}
#             rec['bookID'] = i.find('div', attrs={'u-anchorTarget'}).get('id')
#             rec['bookLink'] = i.find('a', attrs={'class': 'bookTitle'}).get(
#                 'href')
#             rec['bookTitle'] = i.find('a', attrs={'class': 'bookTitle'}).find(
#                 'span').text
#             rec['authorName'] = i.find('a', attrs={'class': 'authorName'}).text
#             rec['award'] = i.find('i').text
#             award_books.append(pd.DataFrame(rec, index=[0]))

#         award_books = pd.concat(award_books)
#         all_awards.append(award_books)
#         logging.info('sleep for a bit')
#         time.sleep(sleepTime)

#     logging.info('finished')
#     all_awards = pd.concat(all_awards)
#     return all_awards

# nebula = getAwardBooks('https://www.goodreads.com/award/show/23-nebula-award',
#                        28, 2)
# nebula.to_csv('/content/drive/My Drive/Colab Notebooks/nebula-awards.csv')
# time.sleep(5)
# goodreads = getAwardBooks(
#     'https://www.goodreads.com/award/show/21332-goodreads-choice-award', 133, 5)
# goodreads.to_csv('/content/drive/My Drive/Colab Notebooks/goodreads-choice-awards.csv')

# time.sleep(5)
# locus = getAwardBooks('https://www.goodreads.com/award/show/46-locus-award', 62,
#                       2)
# locus.to_csv('/content/drive/My Drive/Colab Notebooks/locus-awards.csv')

# time.sleep(5)
# hugo = getAwardBooks('https://www.goodreads.com/award/show/9-hugo-award', 33,
#                       2)
# hugo.to_csv('/content/drive/My Drive/Colab Notebooks/hugo-awards.csv')

# combo = pd.concat([nebula,locus,hugo])
# #combo = combo[~combo['award'].str.contains('Nominee')]
# combo['Nominee']= np.where(combo['award'].str.contains('Nominee'),1,0)
# combo.to_csv('/content/drive/My Drive/Colab Notebooks/combo-awards.csv')
# #combo[combo['award'].str.contains('Nominee')]

## Need to test adding this line to drop the first column with zeros
##awards.drop(awards.columns[0],axis=1,inplace=True)

In [4]:
awards = pd.read_csv('/Users/johnfields/Library/Mobile Documents/com~apple~CloudDocs/Syracuse/IST718/Assignments/Project/UCSD Book Graph/combo-awards.csv')
awards.bookID = awards.bookID.astype(str)
awards.Nominee = awards.Nominee.astype(int)

In [0]:
awards['Nominee_Winner'] = 1
awards

,bookID,bookLink,bookTitle,authorName,award,Nominee,Nominee_Winner
0,43466735,/book/show/43466735-fireside-magazine-issue-52...,"Fireside Magazine Issue 52, February 2018",Julia Rios,"Nebula Award for Short Story for ""The Secret L...",0,1
1,34659272,/book/show/34659272-the-only-harmless-great-thing,The Only Harmless Great Thing,Brooke Bolander,Nebula Award for Best Novelette (2018),0,1
2,36686547,/book/show/36686547-the-tea-master-and-the-det...,The Tea Master and the Detective,Aliette de Bodard,Nebula Award for Novella (2018),0,1
3,33080122,/book/show/33080122-the-calculating-stars,"The Calculating Stars (Lady Astronaut, #1)",Mary Robinette Kowal,Nebula Award for Best Novel (2018),0,1
4,35957452,/book/show/35957452-apex-magazine-issue-99,Apex Magazine Issue 99,Jason Sizemore,"Nebula Award for Best Short Story for ""Welcome...",0,1
...,...,...,...,...,...,...,...
3585,29519390,/book/show/29519390-brightside-crossing,Brightside Crossing,Alan E. Nourse,Hugo Award Nominee for Best Novelette (1956),1,1
3586,2935921,/book/show/2935921-a-gun-for-dinosaur,A Gun for Dinosaur,L. Sprague de Camp,Hugo Award Nominee for Best Novelette (1956),1,1
3587,91109,/book/show/91109.The_Long_Tomorrow,The Long Tomorrow,Leigh Brackett,Hugo Award Nominee for Best Novel (1956),1,1
3588,509784,/book/show/509784.The_End_of_Eternity,The End of Eternity,Isaac Asimov,Hugo Award Nominee for Best Novel (1956),1,1


## 1.1.1 Reviews

In [6]:
print(' == sample record (reviewsFP ==')
display(np.random.choice(reviewsFP))

 == sample record (reviewsFP ==


{'user_id': 'cadf459d4238bc3f3a60ada4ab3fa133',
 'book_id': '15783514',
 'review_id': 'f13f6d9c048781716e41ce2b56b0e6df',
 'rating': 5,
 'review_text': 'Delightful.',
 'date_added': 'Thu Jul 11 02:40:23 -0700 2013',
 'date_updated': 'Sat Jul 13 15:22:29 -0700 2013',
 'read_at': 'Sat Jul 13 15:22:29 -0700 2013',
 'started_at': 'Sat Jul 13 00:00:00 -0700 2013',
 'n_votes': 0,
 'n_comments': 0}

In [7]:
reviewsFPpandas = pd.DataFrame(reviewsFP)

## 1.1.2 BooksFP

In [8]:
booksFPpandas = pd.DataFrame(booksFP)

## 1.1.3 Merge

In [9]:
booksFPreviewsFP = booksFPpandas.merge(reviewsFPpandas, left_on='book_id', right_on='book_id',how="inner")

In [0]:
#booksFPreviewsFP.to_csv('/Users/johnfields/Desktop/booksFPreviewsFP.csv')

In [10]:
booksFPawardsReviewsFPall = booksFPreviewsFP.merge(awards, left_on='book_id', right_on='bookID', how='left')

In [11]:
booksFPawardsReviewsFPall = booksFPawardsReviewsFPall.drop(['series','country_code','language_code','popular_shelves','is_ebook','kindle_asin','similar_books','link','url','image_url','bookLink'],axis=1)

In [0]:
booksFPawardsReviewsFPall.to_csv('/Users/johnfields/Desktop/booksFPawardsReviewsFPall.csv')